In [340]:
import re
import networkx

file = open('data/16.txt', 'r')
data = file.read()

file.close()

In [341]:
rates = {}
G = networkx.DiGraph()

for line in data.splitlines():
    rate = int(re.findall(r'\d+', line).pop())
    valve, *tunnels = re.findall(r'[A-Z]{2}', line)

    rates[valve] = rate
    
    for next in tunnels:
        G.add_edge(valve, next)

In [342]:
import functools
import operator

all_paths = {}

def get_all_paths(start):
    paths = []
    
    for valve in rates:
        if rates[valve] == 0:
            continue

        path = networkx.shortest_path(G, start, valve)
        paths.append(path)

    return paths

def get_pressure(route):
    flow, timer, pressure = 0, 0, 0
    
    for path in route:
        timer += len(path)
        pressure += flow * len(path)
        flow += rates[path[-1]]
        
    pressure += flow * (30 - timer)

    return pressure

In [343]:
pressures = set()

for _, (valve, rate) in enumerate(rates.items()):
    all_paths[valve] = get_all_paths(valve)

def find_routes(current_route, start, open_valves):
    for path in all_paths[start]:
        open_valve = path[-1]

        if open_valve in open_valves:
            continue

        new_route = current_route + [path]

        if len(sum(new_route, [])) > 30:
            continue

        pressures.add(get_pressure(new_route))
        find_routes(new_route, open_valve, open_valves + [open_valve])

find_routes([], 'AA', [])

max(pressures)

1638